In [ ]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback



import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'
projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]

In [3]:
def prepare_data(path):
    df = pd.read_csv(path)
    df = df.drop(labels = ['Host','Vcs','Project','File','PL','IssueTracking'],axis=1)
    df = df.dropna()
    df = df[['TLOC', 'TNF', 'TNC', 'TND', 'LOC', 'CL', 'NStmt', 'NFunc',
       'RCC', 'MNL', 'avg_WMC', 'max_WMC', 'total_WMC', 'avg_DIT', 'max_DIT',
       'total_DIT', 'avg_RFC', 'max_RFC', 'total_RFC', 'avg_NOC', 'max_NOC',
       'total_NOC', 'avg_CBO', 'max_CBO', 'total_CBO', 'avg_DIT.1',
       'max_DIT.1', 'total_DIT.1', 'avg_NIV', 'max_NIV', 'total_NIV',
       'avg_NIM', 'max_NIM', 'total_NIM', 'avg_NOM', 'max_NOM', 'total_NOM',
       'avg_NPBM', 'max_NPBM', 'total_NPBM', 'avg_NPM', 'max_NPM', 'total_NPM',
       'avg_NPRM', 'max_NPRM', 'total_NPRM', 'avg_CC', 'max_CC', 'total_CC',
       'avg_FANIN', 'max_FANIN', 'total_FANIN', 'avg_FANOUT', 'max_FANOUT',
       'total_FANOUT', 'NRev', 'NFix', 'avg_AddedLOC', 'max_AddedLOC',
       'total_AddedLOC', 'avg_DeletedLOC', 'max_DeletedLOC',
       'total_DeletedLOC', 'avg_ModifiedLOC', 'max_ModifiedLOC',
       'total_ModifiedLOC','Buggy']]
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.Buggy.values
    X = df.drop(labels = ['Buggy'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Buggy')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def tune_learner(learner, train_X, train_Y, tune_X, tune_Y, goal,loc=None,target_class=None):
    if not target_class:
        target_class = goal
    clf = learner(train_X, train_Y, tune_X, tune_Y, goal,loc)
    tuner = DE.DE_Tune_ML(clf, clf.get_param(), goal, target_class)
    return tuner.Tune()

In [4]:
final_score = {}
count = 0
for project in projects:
    try:
        path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + project
        print(project)
        df = prepare_data(path)
        if df.shape[0] < 50:
            continue
        else:
            count+=1
        df.reset_index(drop=True,inplace=True)
        d = {'buggy': True, 'clean': False}
        df['Buggy'] = df['Buggy'].map(d)
        buggy = df[df['Buggy'] == True]
        buggy_percentage = buggy.shape[0]/df.shape[0]
        y = df.Buggy
        X = df.drop(labels = ['Buggy'],axis = 1)
        kf = StratifiedKFold(n_splits = 5)
        goal = 'f1'
        F = {}
        score = {}
        for i in range(5):
            for train_index, tune_index in kf.split(X, y):
                X_train, X_tune = X.iloc[train_index], X.iloc[tune_index]
                y_train, y_tune = y[train_index], y[tune_index]
                _df_tune_loc = X_tune.LOC
                #clf = LogisticRegression()
                clf = SVC()
                clf.fit(X_train,y_train)
                predicted = clf.predict(X_tune)
                abcd = metrices.measures(y_tune,predicted,_df_tune_loc)
                F['f1'] = [abcd.calculate_f1_score()]
                F['precision'] = [abcd.calculate_precision()]
                F['recall'] = [abcd.calculate_recall()]
                F['g-score'] = [abcd.get_g_score()]
                F['d2h'] = [abcd.calculate_d2h()]
                F['pci_20'] = [abcd.get_pci_20()]
                F['ifa'] = [abcd.get_ifa()]
                F['pd'] = [abcd.get_pd()]
                F['pf'] = [abcd.get_pf()]
                _F = copy.deepcopy(F)
                print(_F)
                if 'f1' not in score.keys():
                    _F['buggy_prec'] = [buggy_percentage]
                    score = _F
                else:
                    score['f1'].append(F['f1'][0])
                    score['precision'].append(F['precision'][0])
                    score['recall'].append(F['recall'][0])
                    score['g-score'].append(F['g-score'][0])
                    score['d2h'].append(F['d2h'][0])
                    score['pci_20'].append(F['pci_20'][0])
                    score['ifa'].append(F['ifa'][0])
                    score['pd'].append(F['pd'][0])
                    score['pf'].append(F['pf'][0])
                    score['buggy_prec'].append(buggy_percentage)
            final_score[project] = score 
    except Exception as e:
        print(e)
        continue

bzbyte.csv
{'f1': [0.86], 'precision': [0.75], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.31], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.86], 'precision': [0.75], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.31], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.85], 'precision': [0.73], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.27], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.88], 'precision': [0.79], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.29], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.88], 'precision': [0.79], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.29], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.86], 'precision': [0.75], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.31], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.86], 'precision': [0.75], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.31], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
{'

{'f1': [0.33], 'precision': [1.0], 'recall': [0.2], 'g-score': [0.24], 'd2h': [0.8], 'pci_20': [0.29], 'ifa': [0], 'pd': [0.2], 'pf': [0.0]}
{'f1': [0.67], 'precision': [0.75], 'recall': [0.6], 'g-score': [0.64], 'd2h': [0.42], 'pci_20': [0.14], 'ifa': [0], 'pd': [0.6], 'pf': [0.11]}
tauruss.csv
makumba.csv
openi.csv
pure-lang.csv
roguelike.csv
iaml.csv
{'f1': [0.73], 'precision': [0.63], 'recall': [0.86], 'g-score': [0.65], 'd2h': [0.68], 'pci_20': [0.33], 'ifa': [0], 'pd': [0.86], 'pf': [0.67]}
{'f1': [0.69], 'precision': [0.57], 'recall': [0.89], 'g-score': [0.34], 'd2h': [0.91], 'pci_20': [0.33], 'ifa': [1], 'pd': [0.89], 'pf': [0.9]}
{'f1': [0.67], 'precision': [0.6], 'recall': [0.75], 'g-score': [0.6], 'd2h': [0.71], 'pci_20': [0.33], 'ifa': [6], 'pd': [0.75], 'pf': [0.67]}
{'f1': [0.75], 'precision': [0.64], 'recall': [0.89], 'g-score': [0.66], 'd2h': [0.68], 'pci_20': [0.31], 'ifa': [1], 'pd': [0.89], 'pf': [0.67]}
{'f1': [0.75], 'precision': [0.75], 'recall': [0.75], 'g-score'

{'f1': [0.53], 'precision': [0.75], 'recall': [0.41], 'g-score': [0.46], 'd2h': [0.6], 'pci_20': [0.12], 'ifa': [0], 'pd': [0.41], 'pf': [0.08]}
{'f1': [0.68], 'precision': [0.81], 'recall': [0.59], 'g-score': [0.64], 'd2h': [0.42], 'pci_20': [0.14], 'ifa': [0], 'pd': [0.59], 'pf': [0.08]}
{'f1': [0.41], 'precision': [0.64], 'recall': [0.3], 'g-score': [0.35], 'd2h': [0.7], 'pci_20': [0.1], 'ifa': [0], 'pd': [0.3], 'pf': [0.11]}
{'f1': [0.51], 'precision': [0.55], 'recall': [0.48], 'g-score': [0.52], 'd2h': [0.58], 'pci_20': [0.13], 'ifa': [1], 'pd': [0.48], 'pf': [0.24]}
{'f1': [0.58], 'precision': [0.69], 'recall': [0.5], 'g-score': [0.55], 'd2h': [0.52], 'pci_20': [0.14], 'ifa': [3], 'pd': [0.5], 'pf': [0.14]}
{'f1': [0.53], 'precision': [0.75], 'recall': [0.41], 'g-score': [0.46], 'd2h': [0.6], 'pci_20': [0.12], 'ifa': [0], 'pd': [0.41], 'pf': [0.08]}
{'f1': [0.68], 'precision': [0.81], 'recall': [0.59], 'g-score': [0.64], 'd2h': [0.42], 'pci_20': [0.14], 'ifa': [0], 'pd': [0.59], 

{'f1': [0.17], 'precision': [0.5], 'recall': [0.1], 'g-score': [0.12], 'd2h': [0.9], 'pci_20': [0.27], 'ifa': [0], 'pd': [0.1], 'pf': [0.05]}
{'f1': [0.89], 'precision': [1.0], 'recall': [0.8], 'g-score': [0.83], 'd2h': [0.2], 'pci_20': [0.13], 'ifa': [0], 'pd': [0.8], 'pf': [0.0]}
{'f1': [0.2], 'precision': [1.0], 'recall': [0.11], 'g-score': [0.13], 'd2h': [0.89], 'pci_20': [0.45], 'ifa': [0], 'pd': [0.11], 'pf': [0.0]}
{'f1': [0.2], 'precision': [1.0], 'recall': [0.11], 'g-score': [0.13], 'd2h': [0.89], 'pci_20': [0.34], 'ifa': [0], 'pd': [0.11], 'pf': [0.0]}
{'f1': [0.2], 'precision': [1.0], 'recall': [0.11], 'g-score': [0.13], 'd2h': [0.89], 'pci_20': [0.39], 'ifa': [0], 'pd': [0.11], 'pf': [0.0]}
{'f1': [0.17], 'precision': [0.5], 'recall': [0.1], 'g-score': [0.12], 'd2h': [0.9], 'pci_20': [0.27], 'ifa': [0], 'pd': [0.1], 'pf': [0.05]}
{'f1': [0.89], 'precision': [1.0], 'recall': [0.8], 'g-score': [0.83], 'd2h': [0.2], 'pci_20': [0.13], 'ifa': [0], 'pd': [0.8], 'pf': [0.0]}
{'f1'

{'f1': [0.33], 'precision': [0.5], 'recall': [0.25], 'g-score': [0.29], 'd2h': [0.77], 'pci_20': [0.1], 'ifa': [0], 'pd': [0.25], 'pf': [0.17]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.27], 'ifa': [10], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.36], 'ifa': [10], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.05], 'pci_20': [0.1], 'ifa': [9], 'pd': [0.0], 'pf': [0.33]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.4], 'ifa': [9], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.33], 'precision': [0.5], 'recall': [0.25], 'g-score': [0.29], 'd2h': [0.77], 'pci_20': [0.1], 'ifa': [0], 'pd': [0.25], 'pf': [0.17]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.27], 'ifa': [10], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 

...

{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.01], 'pci_20': [0.31], 'ifa': [12], 'pd': [0.0], 'pf': [0.11]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.38], 'ifa': [12], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.27], 'ifa': [14], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.29], 'ifa': [13], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.46], 'ifa': [12], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.01], 'pci_20': [0.31], 'ifa': [12], 'pd': [0.0], 'pf': [0.11]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.38], 'ifa': [12], 'pd': [0.0], 'pf': [0.0]}
amsn.csv
intamap

{'f1': [0.57], 'precision': [1.0], 'recall': [0.4], 'g-score': [0.45], 'd2h': [0.6], 'pci_20': [0.06], 'ifa': [0], 'pd': [0.4], 'pf': [0.0]}
{'f1': [0.57], 'precision': [0.67], 'recall': [0.5], 'g-score': [0.55], 'd2h': [0.51], 'pci_20': [0.07], 'ifa': [0], 'pd': [0.5], 'pf': [0.09]}
{'f1': [0.67], 'precision': [1.0], 'recall': [0.5], 'g-score': [0.56], 'd2h': [0.5], 'pci_20': [0.07], 'ifa': [0], 'pd': [0.5], 'pf': [0.0]}
{'f1': [0.67], 'precision': [1.0], 'recall': [0.5], 'g-score': [0.56], 'd2h': [0.5], 'pci_20': [0.07], 'ifa': [0], 'pd': [0.5], 'pf': [0.0]}
{'f1': [0.8], 'precision': [0.8], 'recall': [0.8], 'g-score': [0.82], 'd2h': [0.22], 'pci_20': [0.12], 'ifa': [0], 'pd': [0.8], 'pf': [0.08]}
{'f1': [0.57], 'precision': [1.0], 'recall': [0.4], 'g-score': [0.45], 'd2h': [0.6], 'pci_20': [0.06], 'ifa': [0], 'pd': [0.4], 'pf': [0.0]}
{'f1': [0.57], 'precision': [0.67], 'recall': [0.5], 'g-score': [0.55], 'd2h': [0.51], 'pci_20': [0.07], 'ifa': [0], 'pd': [0.5], 'pf': [0.09]}
{'f1':

{'f1': [0.84], 'precision': [0.73], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.27], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.84], 'precision': [0.73], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.18], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.89], 'precision': [0.8], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.3], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.88], 'precision': [0.78], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.22], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
apolloplayer.csv
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.33], 'ifa': [20], 'pd': [0.0], 'pf': [0.06]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.43], 'ifa': [20], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.38], 'ifa': [20], 'pd': [0.0], 'pf': [0.0]}


{'f1': [0.96], 'precision': [0.93], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.36], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96], 'precision': [0.92], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.46], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96], 'precision': [0.92], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.31], 'ifa': [1], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96], 'precision': [0.92], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.38], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96], 'precision': [0.93], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.36], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96], 'precision': [0.93], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.36], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96], 'precision': [0.92], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.46], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
{'f1': [0.96]

{'f1': [0.92], 'precision': [0.86], 'recall': [1.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.39], 'ifa': [0], 'pd': [1.0], 'pf': [1.0]}
fw4spl.csv
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.37], 'ifa': [74], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.37], 'ifa': [74], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.41], 'ifa': [73], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.4], 'ifa': [72], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.38], 'ifa': [72], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0.0], 'precision': [0.0], 'recall': [0.0], 'g-score': [0.0], 'd2h': [1.0], 'pci_20': [0.37], 'ifa': [74], 'pd': [0.0], 'pf': [0.0]}
{'f1': [0

In [ ]:
with open('data/1385/1385_LR_default_svm.pkl', 'wb') as handle:
    pickle.dump(final_score, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pd.read_pickle('data/1385/1385_LR_default_svm.pkl')
results = []
for project in df.keys():
    results.append([project,np.median(df[project]['f1']),
                   np.median(df[project]['precision']),
                   np.median(df[project]['recall']),
                   np.median(df[project]['g-score']),
                   np.median(df[project]['d2h']),
                   np.median(df[project]['pci_20']),
                   np.median(df[project]['ifa']),
                   np.median(df[project]['pd']),
                   np.median(df[project]['pf']),
                   np.median(df[project]['buggy_prec'])])

In [ ]:
results = []
for project in df.keys():
    results.append([project,np.median(df[project]['f1']),
                   np.median(df[project]['precision']),
                   np.median(df[project]['recall']),
                   np.median(df[project]['g-score']),
                   np.median(df[project]['d2h']),
                   np.median(df[project]['pci_20']),
                   np.median(df[project]['ifa']),
                   np.median(df[project]['pd']),
                   np.median(df[project]['pf']),
                   np.median(df[project]['buggy_prec'])])

In [ ]:
results_df = pd.DataFrame(results,columns = ['project','f1','precision','recall','g-score','d2h','pci_20','ifa','pd','pf','buggyness'])


In [ ]:
results_df.to_csv('data/1385/1385_LR_default_svm.csv')

In [ ]:
score = np.array([.68,.54,.77,.66,.66,.73,.5,.77,.62,.58,.7,.46,.73,.66,.66,.76,.54,.56,.66,.66,.71,.29,.69,.58,.54])
print(np.median(score))
plt.boxplot(score)